In [9]:
import csv
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import time
from pprint import pprint

In [2]:
version='23_24_gw1'
league_code = "551115"

In [3]:
#API links

#Public GW Stats
#https://fantasy.premierleague.com/api/bootstrap-static/

#Member codes & links
#https://fantasy.premierleague.com/api/entry/######/
     
#Member picks
#https://fantasy.premierleague.com/api/entry/#[MemberID]#/event/#[GW ID]#/picks/


In [4]:
start=time.perf_counter()

In [5]:
#Request league standings and return all league data 
def get_json_from_url(league_code):
    league_url="https://fantasy.premierleague.com/api/leagues-classic/"+league_code+"/standings/?page_new_entries=1&page_standings=1&phase=1"
    get=requests.get(league_url)
    league_data=json.loads(get.text)
    print("Data fetched, league name: ",league_data['league'].get('name'))
    return league_data

#Use league data to return a dict of names & entry ids
def make_ids(league_data):
    member_ids={}
    for member in league_data['standings']['results']:
        name=member.get('player_name')
        mid=member.get('entry')
        member_ids[name]=mid
    return member_ids

#Use first value in member_ids to find the max gameweek that data is available for
def find_max_gw(xid): #xid=member entry id
    gw=1
    while gw < 100:
        try:
            get_mgw=requests.get("https://fantasy.premierleague.com/api/entry/"+str(xid)+"/event/"+str(gw)+"/picks/")
            data_mgw=json.loads(get_mgw.text)
            list(data_mgw['entry_history'].keys())
            gw+=1
        except KeyError as err:
            print("Max gameweek data available for is: ",gw)
            return (gw)
            break

In [6]:
#Doesn't work until GW1!!
league_data = get_json_from_url(league_code)
member_ids=make_ids(league_data)
mgw=find_max_gw(list(member_ids.values())[0])

Data fetched, league name:  RIVERSIDE LEAGUE X 2023/24


IndexError: list index out of range

In [ ]:
#Get data1
def get_first_member_data(member_ids):
    xid=list(member_ids.values())[0]
    gw=1
    get_first=requests.get("https://fantasy.premierleague.com/api/entry/"+str(xid)+"/event/"+str(gw)+"/picks/")
    data_first=json.loads(get_first.text)
    return data_first

#Member picks
def dataframe_from_ids(member_ids,cols):
    p=[]

     #member data

    for xid in member_ids.values():
        for gw in range(1,mgw):    
            get_gw=requests.get("https://fantasy.premierleague.com/api/entry/"+str(xid)+"/event/"+str(gw)+"/picks/")
            data_gw=json.loads(get_gw.text)

            for x in range(0,15):
                #picks
                row=data_gw["picks"][x]

                name=list(member_ids.keys())[list(member_ids.values()).index(xid)]
                selected_row = [name,xid,gw,x+1]

                for i in cols:
                    selected_row.append(row.get(i))

                p.append(selected_row)

                print(xid,name,"GW "+str(gw),"Pick "+str(x+1))
    print("Finished")

    pcols=['member','memberid','gw','pick']
    pcols.extend(cols)

    picksdf = pd.DataFrame(p, columns=pcols)

    return picksdf

data_for_columns=get_first_member_data(member_ids)
cols=list(data_for_columns["picks"][0].keys())
picksdf=dataframe_from_ids(member_ids,cols)

6249286 Robert McGarr GW 1 Pick 1
6249286 Robert McGarr GW 1 Pick 2
6249286 Robert McGarr GW 1 Pick 3
6249286 Robert McGarr GW 1 Pick 4
6249286 Robert McGarr GW 1 Pick 5
6249286 Robert McGarr GW 1 Pick 6
6249286 Robert McGarr GW 1 Pick 7
6249286 Robert McGarr GW 1 Pick 8
6249286 Robert McGarr GW 1 Pick 9
6249286 Robert McGarr GW 1 Pick 10
6249286 Robert McGarr GW 1 Pick 11
6249286 Robert McGarr GW 1 Pick 12
6249286 Robert McGarr GW 1 Pick 13
6249286 Robert McGarr GW 1 Pick 14
6249286 Robert McGarr GW 1 Pick 15
6249286 Robert McGarr GW 2 Pick 1
6249286 Robert McGarr GW 2 Pick 2
6249286 Robert McGarr GW 2 Pick 3
6249286 Robert McGarr GW 2 Pick 4
6249286 Robert McGarr GW 2 Pick 5
6249286 Robert McGarr GW 2 Pick 6
6249286 Robert McGarr GW 2 Pick 7
6249286 Robert McGarr GW 2 Pick 8
6249286 Robert McGarr GW 2 Pick 9
6249286 Robert McGarr GW 2 Pick 10
6249286 Robert McGarr GW 2 Pick 11
6249286 Robert McGarr GW 2 Pick 12
6249286 Robert McGarr GW 2 Pick 13
6249286 Robert McGarr GW 2 Pick 14
624

6265167 Daniel Chadder GW 5 Pick 1
6265167 Daniel Chadder GW 5 Pick 2
6265167 Daniel Chadder GW 5 Pick 3
6265167 Daniel Chadder GW 5 Pick 4
6265167 Daniel Chadder GW 5 Pick 5
6265167 Daniel Chadder GW 5 Pick 6
6265167 Daniel Chadder GW 5 Pick 7
6265167 Daniel Chadder GW 5 Pick 8
6265167 Daniel Chadder GW 5 Pick 9
6265167 Daniel Chadder GW 5 Pick 10
6265167 Daniel Chadder GW 5 Pick 11
6265167 Daniel Chadder GW 5 Pick 12
6265167 Daniel Chadder GW 5 Pick 13
6265167 Daniel Chadder GW 5 Pick 14
6265167 Daniel Chadder GW 5 Pick 15
6265167 Daniel Chadder GW 6 Pick 1
6265167 Daniel Chadder GW 6 Pick 2
6265167 Daniel Chadder GW 6 Pick 3
6265167 Daniel Chadder GW 6 Pick 4
6265167 Daniel Chadder GW 6 Pick 5
6265167 Daniel Chadder GW 6 Pick 6
6265167 Daniel Chadder GW 6 Pick 7
6265167 Daniel Chadder GW 6 Pick 8
6265167 Daniel Chadder GW 6 Pick 9
6265167 Daniel Chadder GW 6 Pick 10
6265167 Daniel Chadder GW 6 Pick 11
6265167 Daniel Chadder GW 6 Pick 12
6265167 Daniel Chadder GW 6 Pick 13
6265167 Da

5223227 Will Hossack GW 4 Pick 1
5223227 Will Hossack GW 4 Pick 2
5223227 Will Hossack GW 4 Pick 3
5223227 Will Hossack GW 4 Pick 4
5223227 Will Hossack GW 4 Pick 5
5223227 Will Hossack GW 4 Pick 6
5223227 Will Hossack GW 4 Pick 7
5223227 Will Hossack GW 4 Pick 8
5223227 Will Hossack GW 4 Pick 9
5223227 Will Hossack GW 4 Pick 10
5223227 Will Hossack GW 4 Pick 11
5223227 Will Hossack GW 4 Pick 12
5223227 Will Hossack GW 4 Pick 13
5223227 Will Hossack GW 4 Pick 14
5223227 Will Hossack GW 4 Pick 15
5223227 Will Hossack GW 5 Pick 1
5223227 Will Hossack GW 5 Pick 2
5223227 Will Hossack GW 5 Pick 3
5223227 Will Hossack GW 5 Pick 4
5223227 Will Hossack GW 5 Pick 5
5223227 Will Hossack GW 5 Pick 6
5223227 Will Hossack GW 5 Pick 7
5223227 Will Hossack GW 5 Pick 8
5223227 Will Hossack GW 5 Pick 9
5223227 Will Hossack GW 5 Pick 10
5223227 Will Hossack GW 5 Pick 11
5223227 Will Hossack GW 5 Pick 12
5223227 Will Hossack GW 5 Pick 13
5223227 Will Hossack GW 5 Pick 14
5223227 Will Hossack GW 5 Pick 1

1490038 Daniel Sadler GW 3 Pick 1
1490038 Daniel Sadler GW 3 Pick 2
1490038 Daniel Sadler GW 3 Pick 3
1490038 Daniel Sadler GW 3 Pick 4
1490038 Daniel Sadler GW 3 Pick 5
1490038 Daniel Sadler GW 3 Pick 6
1490038 Daniel Sadler GW 3 Pick 7
1490038 Daniel Sadler GW 3 Pick 8
1490038 Daniel Sadler GW 3 Pick 9
1490038 Daniel Sadler GW 3 Pick 10
1490038 Daniel Sadler GW 3 Pick 11
1490038 Daniel Sadler GW 3 Pick 12
1490038 Daniel Sadler GW 3 Pick 13
1490038 Daniel Sadler GW 3 Pick 14
1490038 Daniel Sadler GW 3 Pick 15
1490038 Daniel Sadler GW 4 Pick 1
1490038 Daniel Sadler GW 4 Pick 2
1490038 Daniel Sadler GW 4 Pick 3
1490038 Daniel Sadler GW 4 Pick 4
1490038 Daniel Sadler GW 4 Pick 5
1490038 Daniel Sadler GW 4 Pick 6
1490038 Daniel Sadler GW 4 Pick 7
1490038 Daniel Sadler GW 4 Pick 8
1490038 Daniel Sadler GW 4 Pick 9
1490038 Daniel Sadler GW 4 Pick 10
1490038 Daniel Sadler GW 4 Pick 11
1490038 Daniel Sadler GW 4 Pick 12
1490038 Daniel Sadler GW 4 Pick 13
1490038 Daniel Sadler GW 4 Pick 14
149

267919 Conner Harling GW 2 Pick 1
267919 Conner Harling GW 2 Pick 2
267919 Conner Harling GW 2 Pick 3
267919 Conner Harling GW 2 Pick 4
267919 Conner Harling GW 2 Pick 5
267919 Conner Harling GW 2 Pick 6
267919 Conner Harling GW 2 Pick 7
267919 Conner Harling GW 2 Pick 8
267919 Conner Harling GW 2 Pick 9
267919 Conner Harling GW 2 Pick 10
267919 Conner Harling GW 2 Pick 11
267919 Conner Harling GW 2 Pick 12
267919 Conner Harling GW 2 Pick 13
267919 Conner Harling GW 2 Pick 14
267919 Conner Harling GW 2 Pick 15
267919 Conner Harling GW 3 Pick 1
267919 Conner Harling GW 3 Pick 2
267919 Conner Harling GW 3 Pick 3
267919 Conner Harling GW 3 Pick 4
267919 Conner Harling GW 3 Pick 5
267919 Conner Harling GW 3 Pick 6
267919 Conner Harling GW 3 Pick 7
267919 Conner Harling GW 3 Pick 8
267919 Conner Harling GW 3 Pick 9
267919 Conner Harling GW 3 Pick 10
267919 Conner Harling GW 3 Pick 11
267919 Conner Harling GW 3 Pick 12
267919 Conner Harling GW 3 Pick 13
267919 Conner Harling GW 3 Pick 14
267

2556405 Christopher Harris GW 6 Pick 1
2556405 Christopher Harris GW 6 Pick 2
2556405 Christopher Harris GW 6 Pick 3
2556405 Christopher Harris GW 6 Pick 4
2556405 Christopher Harris GW 6 Pick 5
2556405 Christopher Harris GW 6 Pick 6
2556405 Christopher Harris GW 6 Pick 7
2556405 Christopher Harris GW 6 Pick 8
2556405 Christopher Harris GW 6 Pick 9
2556405 Christopher Harris GW 6 Pick 10
2556405 Christopher Harris GW 6 Pick 11
2556405 Christopher Harris GW 6 Pick 12
2556405 Christopher Harris GW 6 Pick 13
2556405 Christopher Harris GW 6 Pick 14
2556405 Christopher Harris GW 6 Pick 15
1913624 Anthony Collier GW 1 Pick 1
1913624 Anthony Collier GW 1 Pick 2
1913624 Anthony Collier GW 1 Pick 3
1913624 Anthony Collier GW 1 Pick 4
1913624 Anthony Collier GW 1 Pick 5
1913624 Anthony Collier GW 1 Pick 6
1913624 Anthony Collier GW 1 Pick 7
1913624 Anthony Collier GW 1 Pick 8
1913624 Anthony Collier GW 1 Pick 9
1913624 Anthony Collier GW 1 Pick 10
1913624 Anthony Collier GW 1 Pick 11
1913624 Ant

882255 Callum Harling GW 4 Pick 1
882255 Callum Harling GW 4 Pick 2
882255 Callum Harling GW 4 Pick 3
882255 Callum Harling GW 4 Pick 4
882255 Callum Harling GW 4 Pick 5
882255 Callum Harling GW 4 Pick 6
882255 Callum Harling GW 4 Pick 7
882255 Callum Harling GW 4 Pick 8
882255 Callum Harling GW 4 Pick 9
882255 Callum Harling GW 4 Pick 10
882255 Callum Harling GW 4 Pick 11
882255 Callum Harling GW 4 Pick 12
882255 Callum Harling GW 4 Pick 13
882255 Callum Harling GW 4 Pick 14
882255 Callum Harling GW 4 Pick 15
882255 Callum Harling GW 5 Pick 1
882255 Callum Harling GW 5 Pick 2
882255 Callum Harling GW 5 Pick 3
882255 Callum Harling GW 5 Pick 4
882255 Callum Harling GW 5 Pick 5
882255 Callum Harling GW 5 Pick 6
882255 Callum Harling GW 5 Pick 7
882255 Callum Harling GW 5 Pick 8
882255 Callum Harling GW 5 Pick 9
882255 Callum Harling GW 5 Pick 10
882255 Callum Harling GW 5 Pick 11
882255 Callum Harling GW 5 Pick 12
882255 Callum Harling GW 5 Pick 13
882255 Callum Harling GW 5 Pick 14
882

In [ ]:
def get_playersummarydf(teams):

    #Playersummarydf = Stats for each player for the season 
    get_pl=requests.get("https://fantasy.premierleague.com/api/bootstrap-static/")
    data_pl=json.loads(get_pl.text)
    cols=list(data_pl['elements'][0].keys())
    d = []
    #Get max number of elemnts
    mpl=[] #max player list
    for i in range(0,len(data_pl['elements'])):
        mpl.append(data_pl['elements'][i].get('id'))
    max_=max(mpl)

    #each row is a player
    #each column is a variable (column list = keylist)

    for x in range(0,max_):
        row=data_pl["elements"][x]

        selected_row = []
        for item in cols:
            selected_row.append(row.get(item))
        d.append(selected_row)
    playersummarydf = pd.DataFrame(d, columns=cols)
    
    teamdf=pd.DataFrame(teams)
    playersummarydf=pd.merge(playersummarydf,teamdf,how='left',on='team')
    
    #Team info
    t=[]
    cols=list(data_pl['teams'][0].keys())   

    for x in range(0,20):
        row=data_pl['teams'][x]

        selected_row = []
        for item in cols:
            selected_row.append(row.get(item))
        t.append(selected_row)
    teamsummarydf=pd.DataFrame(t,columns=cols)
    
    playersummarydf=pd.merge(playersummarydf,teamsummarydf,left_on='team',right_on='id',suffixes=('_player','_team'))
    
    return playersummarydf

#Adding 22/23 team names to playersummarydf (according to the team #)
teams = {'team':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
        ,'team_name':['Arsenal', 'Aston Villa','Bournemouth','Brentford','Brighton'
                      ,'Chelsea', 'Crystal Palace', 'Everton','Fulham', 'Leicester','Leeds'
                      , 'Liverpool', 'Man City', 'Man Utd', 'Newcastle','Nottingham Forest'
                      , 'Southampton', 'Spurs', 'West Ham', 'Wolves']}


playersummarydf=get_playersummarydf(teams)

In [ ]:
#Player details dataframe

def get_player_details():
    #Get a list of columns for player details dataframe
    playerdetailurl="https://fantasy.premierleague.com/api/element-summary/1/"
    get=requests.get(playerdetailurl)
    p_detail_data=json.loads(get.text)      
    cols=list(p_detail_data['history'][0].keys())

    #Get data for each player, for each gameweek, for each column
    bootstrapurl="https://fantasy.premierleague.com/api/bootstrap-static/"
    get=requests.get(bootstrapurl)
    bootstrapdata=json.loads(get.text)

    #Get mp, max player
    mpl=[] #max player list
    for i in range(0,len(bootstrapdata['elements'])):
        mpl.append(bootstrapdata['elements'][i].get('id'))
    mp=max(mpl)

    e=[] #exceptions 1
    e1=[] #exceptions 2

    playerdetails=[]

    for playerid in range(1,mp+1):  
        try:
            print(playerid)
            playerdetailurl="https://fantasy.premierleague.com/api/element-summary/"+str(playerid)+"/"
            get=requests.get(playerdetailurl)
            p_detail_data=json.loads(get.text)

            for gw in range(0,39):    
                try:
                    row=p_detail_data['history'][gw]
                    r=[]
                    for col in cols:
                        r.append(row.get(col))
                    playerdetails.append(r)
                except IndexError as ex:
                    e.extend([playerid, gw,ex])    
        except Exception as ex1:
            e1.extend([playerid,gw,ex1])
    return playerdetails,cols
            

def make_playerdf2(playerdetails,cols):    
    
    playerdf=pd.DataFrame(playerdetails,columns=cols)
    #Adding opponent team names according to the opponent team code
    opponentteams=teams.copy()
    opponentteams['opponent_team'] = opponentteams.pop('team')
    opponentteams['opp_team_name'] = opponentteams.pop('team_name')
    opponentteamdf=pd.DataFrame(opponentteams)
    playerdf=pd.merge(playerdf,opponentteamdf,how='left', on='opponent_team')

    #Rename id to element & merge playerdf/playersummarydf
    playersummarydf.rename(columns={"id_player": "element"}, inplace=True)
    playerdf2=pd.merge(playerdf, playersummarydf,how='left',on='element',suffixes=('_gw', '_summary'))
    return playerdf2

playerdetails,cols=get_player_details()
playerdf2=make_playerdf2(playerdetails,cols)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
def make_alldata(picksdf,playerdf2):
    #alldata = picksdf + playerdf2

    picksdf['player_gw']=picksdf['element'].astype(str)+"_"+picksdf['gw'].astype(str)
    playerdf2['player_gw']=playerdf2['element'].astype(str)+"_"+playerdf2['round'].astype(str)
    alldata=pd.merge(picksdf, playerdf2, how='outer',on='player_gw',suffixes=('_picks', '_player'))
    alldata['gw_total']=alldata['total_points_gw']*alldata['multiplier']

    #All picks chosen by all members in all gameweeks (including unpicked players)
    alldata.drop_duplicates(keep=False,inplace=True)
    print(alldata.shape)
    
    return alldata

alldata=make_alldata(picksdf,playerdf2)

(4671, 131)


In [ ]:
#take out points gained from players in first draft
    
draftpicks=[]
notdraft=[]

for i,m in enumerate(list(member_ids.keys())):
    filtered1=(alldata['gw']==1)
    filtered2=(alldata['member']==m)
    gw1=alldata[['member','gw','pick','element_picks','web_name','gw_total']][filtered1 & filtered2]
    draftpicks.append([m,list(gw1['element_picks'])])
    gws=alldata[['member','gw','pick','element_picks','web_name','gw_total']][filtered2]
    notdraft.append(gws[~gws['element_picks'].isin(draftpicks[i][1])])

nodraft = pd.concat(notdraft) #list of dfs
nodraft

def draftpick(alldata):
    try:
        m=alldata['member']
        e=alldata['element_picks']

        d=[]
        for i in draftpicks:d.append(i[0])
        i=d.index(m)

        if e in draftpicks[i][1]:
            return 1 
        else: 
            return 0
    except ValueError:
        return np.nan
    
alldata['gw1_pick']=alldata.apply(draftpick,axis=1)

membertopn=[]

for n in range(1,11):    
    topnplayers=[]
    for i,m in enumerate(list(member_ids.keys())):
        filtered=(alldata['member']==m)
        best=alldata[['member','gw','element_picks','gw_total']]
        best_f=best[filtered].groupby(['element_picks'],as_index=False).sum().sort_values(by=['gw_total'], ascending=False)
        topn=best_f[:n]
        topnplayers.append([m,list(topn['element_picks'])])
    membertopn.append([n,topnplayers])
    
mdict={}
for i,j in membertopn:
    mdict[i] = {name:(topn) for name, topn in j}

def topnplayer(x,n):
#     n=1
    for m in list(member_ids.keys()):
        if x['member']==m:
            tn=set(mdict.get(n).get(m))
            if x['element_picks'] in tn:
                return 1
            elif x['element_picks'] not in tn:
                return 0
            else:return np.nan
        else: continue

alldata['top1']=alldata[['member','element_picks']].apply(topnplayer,n=1,axis=1)
alldata['top2']=alldata[['member','element_picks']].apply(topnplayer,n=2,axis=1)
alldata['top3']=alldata[['member','element_picks']].apply(topnplayer,n=3,axis=1)
alldata['top4']=alldata[['member','element_picks']].apply(topnplayer,n=4,axis=1)
alldata['top5']=alldata[['member','element_picks']].apply(topnplayer,n=5,axis=1)

In [ ]:
def formifier(dfx,var,aggfunc):
    dfslice=dfx[['member','gw','pick', var]]
    dfslice=dfslice.sort_values(['member','gw','pick'], ascending = (True, True, True))
    
    grouped=dfslice.groupby(['member','gw'])
    totals_bygw=grouped.agg('sum')

    formcols=[[] for i in range(0,21)]
    formcolnames=[str(var)+'_form0',str(var)+'_form1',str(var)+'_form2',str(var)+'_form3'
                  ,str(var)+'_form4',str(var)+'_form5',str(var)+'_form6',str(var)+'_form7'
                  ,str(var)+'_form8',str(var)+'_form9',str(var)+'_form10',str(var)+'_form11'
                  ,str(var)+'_form12',str(var)+'_form13',str(var)+'_form14',str(var)+'_form15'
                  ,str(var)+'_form16',str(var)+'_form17',str(var)+'_form18',str(var)+'_form19'
                  ,str(var)+'_form20']

    for i in range(0,21):
        formcols[i]=totals_bygw.groupby(level=0).apply(lambda x: x.rolling(window=i).mean())[var]

    form=pd.DataFrame(formcols).T
    form.columns=formcolnames
    
    gwform=pd.concat([totals_bygw.drop(columns=['pick']),form],axis=1)
    
    return gwform

#Make these variables, using data from picksdf:
    #owned by this member this gw?
        #owned by this member ever?
    #owned in div1 in this gameweek?
    #owned in div2 in this gameweek?
    #available this week?
    
goalsform=formifier(alldata,'goals_scored_gw','sum')
assistsform=formifier(alldata,'assists_gw','sum')
pointsform=formifier(alldata,'gw_total','sum')
    #Merge with gameweek2

Gameweek Level Analysis

In [ ]:
#Optimum gameweek

def formationmax(pickpoints,gw,m):
    #list of formations possible
    formations_possible=[[1,3,5,2],[1,3,4,3],[1,4,5,1],[1,4,4,2],[1,4,3,3],[1,5,4,1],[1,5,3,2],[1,5,2,3]]
    formation_names=['352','343','451','442','433','541','532','523']
    
    f=[]
    flist=[]
    fscore=[]
    bestformation=[]
    bestformationlist=[]
    best_of_each_formation=[]
    
    for count,formation in enumerate(formations_possible):
        typescore=[]
        bestformation=[]
        for p_type in range(1,5):
            typenum=formations_possible[count][p_type-1]
            membergwtype=pickpoints[['member','gw','pick','element_type','element_picks','web_name','element_type','total_points_gw']][(pickpoints['gw']==gw) & (pickpoints['member']==m) & (pickpoints['element_type']==p_type)].sort_values(by='total_points_gw',ascending=False)
            best_in_type=membergwtype[['element_type','element_picks','web_name','total_points_gw']][:typenum]
            pick=list(best_in_type['web_name'].values)
            points=list(best_in_type['total_points_gw'].values)
            best_from_type=list(zip(pick,points))
            bestformation.extend(best_from_type)
            #for this player type in the form [id,points], sum points:
            s = [i[1] for i in best_from_type]     
            typescore.extend([sum(s)])             
        formationscore=sum(typescore)        
        captain=sorted(bestformation, key = lambda f: f[1],reverse=True)[0]        
        totalscore=formationscore+captain[1]       
        best_of_each_formation.append([gw,m,formation_names[count],totalscore,captain,formationscore,bestformation])
        
    formtn_max=sorted(best_of_each_formation, key = lambda x: x[3])[-1] #get total score)
    return formtn_max

def get_optimumdf(alldata):

    fdf=[]
    pickpoints=alldata[['member','gw','pick','element_type','element_picks','web_name','total_points_gw']]

    for m in list(member_ids.keys()):
        for gw in range(1,38+1):
            try:
                x=formationmax(pickpoints,gw,m) #function defined above
                fdf.extend([x])
                print(m,gw)
            except IndexError as err:
                print(m,gw,err)
                continue

    optimumdf=pd.DataFrame(fdf,columns=['gw','member','formation_name','best_score','captain','formationscore','formation_lineup'])
    return optimumdf

optimumdf=get_optimumdf(alldata)

Robert McGarr 1
Robert McGarr 2
Robert McGarr 3
Robert McGarr 4
Robert McGarr 5
Robert McGarr 6
Robert McGarr 7 list index out of range
Robert McGarr 8 list index out of range
Robert McGarr 9 list index out of range
Robert McGarr 10 list index out of range
Robert McGarr 11 list index out of range
Robert McGarr 12 list index out of range
Robert McGarr 13 list index out of range
Robert McGarr 14 list index out of range
Robert McGarr 15 list index out of range
Robert McGarr 16 list index out of range
Robert McGarr 17 list index out of range
Robert McGarr 18 list index out of range
Robert McGarr 19 list index out of range
Robert McGarr 20 list index out of range
Robert McGarr 21 list index out of range
Robert McGarr 22 list index out of range
Robert McGarr 23 list index out of range
Robert McGarr 24 list index out of range
Robert McGarr 25 list index out of range
Robert McGarr 26 list index out of range
Robert McGarr 27 list index out of range
Robert McGarr 28 list index out of range
Rober

Will Hossack 36 list index out of range
Will Hossack 37 list index out of range
Will Hossack 38 list index out of range
Michael Irvine 1
Michael Irvine 2
Michael Irvine 3
Michael Irvine 4
Michael Irvine 5
Michael Irvine 6
Michael Irvine 7 list index out of range
Michael Irvine 8 list index out of range
Michael Irvine 9 list index out of range
Michael Irvine 10 list index out of range
Michael Irvine 11 list index out of range
Michael Irvine 12 list index out of range
Michael Irvine 13 list index out of range
Michael Irvine 14 list index out of range
Michael Irvine 15 list index out of range
Michael Irvine 16 list index out of range
Michael Irvine 17 list index out of range
Michael Irvine 18 list index out of range
Michael Irvine 19 list index out of range
Michael Irvine 20 list index out of range
Michael Irvine 21 list index out of range
Michael Irvine 22 list index out of range
Michael Irvine 23 list index out of range
Michael Irvine 24 list index out of range
Michael Irvine 25 list in

Conner Harling 35 list index out of range
Conner Harling 36 list index out of range
Conner Harling 37 list index out of range
Conner Harling 38 list index out of range
Anthony Powell 1
Anthony Powell 2
Anthony Powell 3
Anthony Powell 4
Anthony Powell 5
Anthony Powell 6
Anthony Powell 7 list index out of range
Anthony Powell 8 list index out of range
Anthony Powell 9 list index out of range
Anthony Powell 10 list index out of range
Anthony Powell 11 list index out of range
Anthony Powell 12 list index out of range
Anthony Powell 13 list index out of range
Anthony Powell 14 list index out of range
Anthony Powell 15 list index out of range
Anthony Powell 16 list index out of range
Anthony Powell 17 list index out of range
Anthony Powell 18 list index out of range
Anthony Powell 19 list index out of range
Anthony Powell 20 list index out of range
Anthony Powell 21 list index out of range
Anthony Powell 22 list index out of range
Anthony Powell 23 list index out of range
Anthony Powell 24 l

Rachel Aumayer 21 list index out of range
Rachel Aumayer 22 list index out of range
Rachel Aumayer 23 list index out of range
Rachel Aumayer 24 list index out of range
Rachel Aumayer 25 list index out of range
Rachel Aumayer 26 list index out of range
Rachel Aumayer 27 list index out of range
Rachel Aumayer 28 list index out of range
Rachel Aumayer 29 list index out of range
Rachel Aumayer 30 list index out of range
Rachel Aumayer 31 list index out of range
Rachel Aumayer 32 list index out of range
Rachel Aumayer 33 list index out of range
Rachel Aumayer 34 list index out of range
Rachel Aumayer 35 list index out of range
Rachel Aumayer 36 list index out of range
Rachel Aumayer 37 list index out of range
Rachel Aumayer 38 list index out of range
Tajwar Shelim 1
Tajwar Shelim 2
Tajwar Shelim 3
Tajwar Shelim 4
Tajwar Shelim 5
Tajwar Shelim 6
Tajwar Shelim 7 list index out of range
Tajwar Shelim 8 list index out of range
Tajwar Shelim 9 list index out of range
Tajwar Shelim 10 list index 

In [ ]:
#in development
# def cap_points_missed(team,captain):

#     highest=sorted(team, key = lambda x: x[1],reverse=True)[0] #total_points_gw

#     if highest[1]>captain[1]:
#         multiplier = 3
#         points_missed = (multiplier-1)*(highest[1] - captain[1])
#         return points_missed
#     else:
#         return 0
    
# team=[["A",10],["B",4],["C",2]] #total_points_gw
# captain=["B",2] #chosen by member, total_points_gw without multipler

# cap_points_missed(team,captain)

In [ ]:
#take out points gained from players in first draft
    
draftpicks=[]
notdraft=[]

for i,m in enumerate(list(member_ids.keys())):
    filtered1=(alldata['gw']==1)
    filtered2=(alldata['member']==m)
    gw1=alldata[['member','gw','pick','element_picks','web_name','gw_total']][filtered1 & filtered2]
    draftpicks.append([m,list(gw1['element_picks'])])
    #print(draftpicks[i])
    gws=alldata[['member','gw','pick','element_picks','web_name','gw_total']][filtered2]
    
    notdraft.append(gws[~gws['element_picks'].isin(draftpicks[i][1])])

nodraft = pd.concat(notdraft) #list of dfs
nodraft

,member,gw,pick,element_picks,web_name,gw_total
284,Robert McGarr,3.0,6.0,313.0,Zinchenko,6.0
363,Robert McGarr,3.0,14.0,211.0,Reed,0.0
485,Robert McGarr,4.0,13.0,313.0,Zinchenko,0.0
489,Robert McGarr,4.0,14.0,211.0,Reed,0.0
537,Robert McGarr,5.0,6.0,211.0,Reed,1.0
...,...,...,...,...,...,...
1694,Tajwar Shelim,3.0,8.0,104.0,Groß,6.0
1697,Tajwar Shelim,4.0,9.0,104.0,Groß,11.0
1701,Tajwar Shelim,5.0,4.0,108.0,Webster,1.0
1705,Tajwar Shelim,6.0,4.0,108.0,Webster,1.0


In [ ]:
# #shift

# def shifted_cols(df,col,periodlist):
#     shiftedcols={}
    
#     for period in periodlist:
#         shifted=df.groupby(level=0).shift(periods=period)[col]
#         colname=str(col)+"_shifted_"+str(period)  
#         shiftedcols[colname]=shifted
#     return shiftedcols

# gameweek=pd.DataFrame()
# df=gameweek

# col='gw_total'
# shiftedcols=shifted_cols(df,col,[0,1,3,5,10,-1,-3,-5,-10])
# dfshifted_points = pd.DataFrame(data=shiftedcols)

# col='gw_total_form3'
# shiftedcols=shifted_cols(df,col,[3,-3])
# dfshifted_form3 = pd.DataFrame(data=shiftedcols)

# col='gw_total_form5'
# shiftedcols=shifted_cols(df,col,[5,-5])
# dfshifted_form5 = pd.DataFrame(data=shiftedcols)

# col='gw_total_form10'
# shiftedcols=shifted_cols(df,col,[10,-10])
# dfshifted_form10 = pd.DataFrame(data=shiftedcols)

In [ ]:
#Merge all gameweek data
gameweek=alldata[['member','gw','gw_total']].groupby(['member','gw'],as_index=False).sum()
gameweek['member_gw'] = gameweek[['member','gw']].apply(tuple, axis=1)

optimumdf['member_gw'] = optimumdf[['member','gw']].apply(tuple, axis=1)

no_draft=nodraft[['member','gw','gw_total']].groupby(['member','gw'],as_index=False).sum().fillna(0)
no_draft['member_gw']=no_draft[['member','gw']].apply(tuple, axis=1)

#Merge
gameweek=gameweek.merge(optimumdf[['member_gw','formation_name','best_score','captain','formationscore','formation_lineup']], how='outer', on='member_gw',suffixes=('_gwdata', '_optimum'))
gameweek=gameweek.merge(no_draft[['member_gw','gw_total']], how='outer', on='member_gw',suffixes=('', '_no_gw1pick'))

# gameweek=pd.merge(gameweek,dfshifted_points,on=['member','season_gw'])
# gameweek=pd.merge(gameweek,dfshifted_form3,on=['member','season_gw'])
# gameweek=pd.merge(gameweek,dfshifted_form5,on=['member','season_gw'])
# gameweek=pd.merge(gameweek,dfshifted_form10,on=['member','season_gw'])

In [ ]:
#Member Gameweeks
dfcsv=gameweek
dfcsv.to_csv(r'C:\Users\chris\Documents\Coding\python\projects\FPL\FPL_data_ouput\DIVX_gameweek'+str(version)+'.csv')
dfcsv.to_csv(r'C:\Users\chris\Documents\Coding\python\projects\FPL\FPL_data_ouput\DIVX_gameweek_updated.csv')

#Member Picks
dfcsv=alldata
dfcsv.to_csv(r'C:\Users\chris\Documents\Coding\python\projects\FPL\FPL_data_ouput\DIVX_picks'+str(version)+'.csv')
dfcsv.to_csv(r'C:\Users\chris\Documents\Coding\python\projects\FPL\FPL_data_ouput\DIVX_picks_updated.csv')

# #all premier league player stats
# dfcsv=playerdf2
# dfcsv.to_csv(r'C:\Users\chris\Documents\Coding\python\projects\FPL\FPL_data_ouput\playergw_'+str(version)+'.csv')
# dfcsv.to_csv(r'C:\Users\chris\Documents\Coding\python\projects\FPL\FPL_data_sources\playergw_'+str(version)+'.csv') #to be used in the xPoints Predictor

# #playersummary stats as of that gameweek
# dfcsv=playersummarydf
# dfcsv.to_csv(r'C:\Users\chris\Documents\Coding\python\projects\FPL\FPL_data_ouput\playersummary'+str(version)+'.csv')

In [ ]:
finish=time.perf_counter()
print(f'Finished in {round(((finish-start)/60),2)} minute(s)')
    
#14/08/22: Finished in 1.9 minute(s)

Finished in 6.99 minute(s)


In [ ]:
#gameweekdata2=gameweekdata2.merge(division19_20,how='outer',on='member')

#merge formifier data #goalsform & #assistsform to this dataframe
    #make points and points form relative to the average
    
#Running total of overall points 
    #for in range(1,47):
        #return sum gw totals, including only gws1 to i
#Divison & League rank overall   
    #get gw totals at gw i
    #find index of this member in this list for this gw.

#At the moment, it merges current player summaries, rather than player summaries as of that gameweek
#In future, further add on summaries as of gw 'X' by searching the FPL folder:
#at gameweek X, append the data from the file 'playersummary20_21_gwX'
#summarise daata from alldata (combine all picks for a gw) to give gameweek level player data eg. goals scored each gw